# Human Activity Recognition Using WiFi Signals

## Overview
Human Activity Recognition (HAR) using WiFi signals leverages the unique properties of wireless channel variations to detect different activities.

## Data Format
- **WiFi signal data** is similar to image data in structure, represented in the shape `(channels, height, width)`, but with a different interpretation:
  - `channels` → **channel**
  - `height` → **Time Steps**
  - `width` → **Antenna Pairs (transmitter-receiver combinations)**
- **Labels** represent a predefined set of classes, as is typical in classification tasks.

# Reading Data

In [1]:
import kagglehub
path = kagglehub.dataset_download("alihabibullah/question-2-data")

print("Path to dataset files:", path)

100%|██████████| 204M/204M [00:01<00:00, 145MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/alihabibullah/question-2-data/versions/1


In [2]:
import os
os.listdir(path)

['WiFiSensingDataset.pt']

In [3]:
import torch

# Load the .pt file
data = torch.load(f"{path}/WiFiSensingDataset.pt")


<ipython-input-3-4d2815cbd2c7>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(f"{path}/WiFiSensingDataset.pt")


# Task 1: Analyze the Dataset ( Stored in `data`)

1. **Determine the number of unique labels** in the dataset.  

2. **Determine the shape of the input data** (number of samples and features).  

3. **Find the maximum value** in the dataset.  

4. **Find the minimum value** in the dataset.  

In [4]:
# 1. Determine the number of unique labels in y_train
number_of_labels = len(torch.unique(data['y_train']))
print(f"Number of unique labels: {number_of_labels}")

# 2. Determine the shape of the input data (X_train)
data_shape = data['X_train'].shape
print(f"Shape of the input data (X_train): {data_shape} (n_samples, channels, time_steps, subcarriers)")

# 3. Find the maximum value in the dataset (X_train)
maximum_value = torch.max(data['X_train'])
print(f"Maximum value in X_train: {maximum_value}")

# 4. Find the minimum value in the dataset (X_train)
minimum_value = torch.min(data['X_train'])
print(f"Minimum value in X_train: {minimum_value}")

Number of unique labels: 7
Shape of the input data (X_train): torch.Size([2500, 1, 250, 90]) (n_samples, channels, time_steps, subcarriers)
Maximum value in X_train: 1.0
Minimum value in X_train: 0.0


# Task 2: Build and Evaluate a Neural Network

1. **Design a Neural Network (Maximum 5 Layers)**  
   Build a compact neural network with no more than 5 layers. Clearly specify the type of each layer (e.g., Dense) and any activation functions used.

2. **Evaluate Your Model**  
   Train your network on the provided dataset and report the evaluation metrics (e.g., accuracy, loss). Discuss the performance of your model and any challenges faced during training.


In [10]:
# Task 2: Build and Evaluate a Neural Network

import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Create a simple neural network (maximum 5 layers)
class SimpleNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(SimpleNN, self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        return self.model(x)

# Prepare the dataset
X_train = data['X_train'].reshape(data['X_train'].shape[0], -1)  # Flatten for fully connected layers
y_train = data['y_train'].long()
X_test = data['X_test'].reshape(data['X_test'].shape[0], -1)
y_test = data['y_test'].long()

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Initialize the model
input_size = X_train.shape[1]
num_classes = number_of_labels
model = SimpleNN(input_size, num_classes)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 10
for epoch in range(epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

# Evaluate the model
with torch.no_grad():
    test_outputs = model(X_test)
    predictions = torch.argmax(test_outputs, axis=1)
    accuracy = (predictions == y_test).sum().item() / y_test.size(0)

print(f"Test Accuracy: {accuracy * 100:.2f}%")

Epoch 1/10, Loss: 2.5785
Epoch 2/10, Loss: 0.8548
Epoch 3/10, Loss: 0.8127
Epoch 4/10, Loss: 0.9009
Epoch 5/10, Loss: 1.4779
Epoch 6/10, Loss: 1.3959
Epoch 7/10, Loss: 1.8243
Epoch 8/10, Loss: 0.6906
Epoch 9/10, Loss: 3.1787
Epoch 10/10, Loss: 0.4917
Test Accuracy: 42.60%


Good luck in the exam x)

Prepared by: Ahmed Y. Radwan
